In [4]:
import h5py
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from nilearn import datasets, plotting

from pathlib import Path
import os
import pandas as pd

In [ ]:
h5_key = 'latent'
## Define paths
basedir = Path('..')
datadir = basedir / 'data'
rawdir = datadir / 'raw'
gccadir = datadir / 'gcca'

## Load Files

In [ ]:
def get_files(path,
              level='(e|n)',
              subject='[0-9]{3}',
              task='.+?',
              filetype='h5',
              flag=''):
    files = []
    query = f'^{level}_sub-'
    query += f'({subject})_ses-1_'
    query += f'task-{task}{flag}\.{filetype}'
    for f in os.listdir(path):
        match = re.search(query, f)
        if match:
            files.append((f, match.group(1)))
    
    return(files)

In [2]:
# plot on fsa5 surface:
fsa5 = datasets.fetch_surf_fsaverage()

In [ ]:
## Get filenames for each task, novice vs. experienced
tasks = ['restingstate', 'openmonitoring', 'compassion']
levels = ['e', 'n']


## Load a single set of latents
#for task,level in np.array(np.meshgrid(tasks,levels)).T.reshape(-1,2):
level=levels[0]
task=tasks[0]
paths = get_files(path=gccadir, level=level, task=task, flag='_gcca')
latents = []
subjs = []

n_load = 1

for path,subj in paths[:n_load]:
    h5f = h5py.File(path,'r')
    latent = h5f[h5_key][:]
    h5f.close()
    latents.append(latent)
    subjs.append(subj)

In [ ]:
embedding = latents[0]

fig, axes = plt.subplots(4, 4, subplot_kw={'projection': '3d'}, figsize=(20, 20))

for i in range(4):
    ax = axes[i]
    X = embeddings[:, i]
    

    cortex_left = nib.freesurfer.read_label('lh.cortex.label')

    conn_map = np.zeros(10242)
    conn_map[cortex_left] = X[:len(cortex_left)]

    plotting.plot_surf_stat_map(fsa5.infl_left, 
      conn_map, hemi='left', colorbar=False, cmap='jet', axes = ax[0])

    plotting.plot_surf_stat_map(fsa5.infl_left,
      conn_map, hemi='left', colorbar=False, cmap='jet', view='medial', axes=ax[1])

    cortex_right = nib.freesurfer.read_label('rh.cortex.label')

    conn_map = np.zeros(10242)
    conn_map[cortex_right] = X[len(cortex_left):]

    plotting.plot_surf_stat_map(fsa5.infl_right, 
      conn_map, hemi='right', colorbar=False, cmap='jet', axes = ax[2])

    plotting.plot_surf_stat_map(fsa5.infl_right,
      conn_map, hemi='right', colorbar=True, cmap='jet', view='medial', axes=ax[3])
